In [ ]:
!git clone https://github.com/MTG/freesound-python.git

Cloning into 'freesound-python'...
remote: Enumerating objects: 417, done.
remote: Counting objects: 100% (125/125), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 417 (delta 58), reused 100 (delta 53), pack-reused 292 (from 1)
Receiving objects: 100% (417/417), 206.37 KiB | 9.83 MiB/s, done.
Resolving deltas: 100% (200/200), done.


In [ ]:
!pip install ./freesound-python

Processing ./freesound-python
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for freesound-python: filename=freesound_python-1.1-py3-none-any.whl size=6496 sha256=1df77d6f0f1320691dc8b455e32338596c320bc4abd95c61c0e5eade9b8726c1
  Stored in directory: /root/.cache/pip/wheels/80/e8/a3/3e192d339b115e258b361c28f8b0e0cd4e54ece6b15a459836
Successfully built freesound-python
  Attempting uninstall: freesound-python
    Found existing installation: freesound-python 0.1
    Uninstalling freesound-python-0.1:
      Successfully uninstalled freesound-python-0.1


In [ ]:
import freesound
print("Freesound client imported successfully!")

Freesound client imported successfully!


In [ ]:
# VIOLENCE DETECTION: Violent Audio Dataset Scraper for Google Colab
# FINAL WORKING VERSION - Fixed preview URL access


# STEP 1: Install packages
print("📦 Installing packages...")
!pip install freesound-python requests tqdm soundfile librosa -q
!apt-get install -y ffmpeg > /dev/null 2>&1

print("✅ Packages installed\n")

# STEP 2: Import everything
import os
import time
import json
import requests
from pathlib import Path
from tqdm import tqdm
import random
from datetime import datetime
import pandas as pd

import freesound

print("✅ All modules imported successfully!\n")

# ============================================================================
# Configuration
# ============================================================================

BASE_PATH = '/content/datasets/violent_audio'
os.makedirs(BASE_PATH, exist_ok=True)

print(f"✓ Dataset location: {BASE_PATH}\n")

# ⚠️ REPLACE THIS WITH YOUR ACTUAL API KEY
FREESOUND_API_KEY = "JmeBmVL3M0FsXnUT7MBTbw2EYXU19e2MIEkw8lRE"

# ============================================================================
# Violent Audio Categories
# ============================================================================

VIOLENT_AUDIO_CATEGORIES = {
    "screams_human_distress": {
        "queries": ["scream", "screaming", "yell", "distress"],
        "target_per_query": 50,
        "duration_range": (1, 5),
    },
    "gunshots_firearms": {
        "queries": ["gunshot", "gun", "pistol", "rifle"],
        "target_per_query": 50,
        "duration_range": (0.5, 3),
    },
    "glass_breaking": {
        "queries": ["glass", "shatter", "window", "bottle"],
        "target_per_query": 40,
        "duration_range": (1, 3),
    },
    "explosions_blasts": {
        "queries": ["explosion", "blast", "bomb", "detonation"],
        "target_per_query": 40,
        "duration_range": (2, 5),
    },
    "sirens_alarms": {
        "queries": ["siren", "alarm", "emergency", "warning"],
        "target_per_query": 40,
        "duration_range": (3, 10),
    },
    "fighting_impact": {
        "queries": ["punch", "impact", "fight", "hit"],
        "target_per_query": 40,
        "duration_range": (0.5, 2),
    },
    "aggressive_speech": {
        "queries": ["shouting", "yelling", "angry", "argument"],
        "target_per_query": 25,
        "duration_range": (2, 10),
    },
    "crash_collision": {
        "queries": ["crash", "collision", "accident", "car"],
        "target_per_query": 25,
        "duration_range": (2, 5),
    }
}

# ============================================================================
# Scraper Class - FINAL FIXED VERSION
# ============================================================================

class ViolentAudioScraper:
    def __init__(self, api_key, base_path=BASE_PATH):
        self.api_key = api_key
        self.base_path = base_path
        self.client = freesound.FreesoundClient()
        self.client.set_token(api_key, "token")

        self.stats = {
            'successful_downloads': 0,
            'failed_downloads': 0,
            'skipped_duplicates': 0,
            'collection_start': datetime.now().isoformat(),
            'collection_end': None,
            'categories': {},
            'licensing_info': []
        }
        self.downloaded_ids = set()

    def search_sounds(self, query, max_results=50, duration_range=(1, 10)):
        """Search using REST API directly"""
        try:
            duration_filter = f"duration:[{duration_range[0]} TO {duration_range[1]}]"

            url = "https://freesound.org/apiv2/search/text/"
            params = {
                "query": query,
                "filter": duration_filter,
                "sort": "rating_desc",
                "fields": "id,name,previews,duration,license,username",
                "page_size": max_results,
                "token": self.api_key
            }

            response = requests.get(url, params=params, timeout=10)
            response.raise_for_status()
            data = response.json()

            # Convert JSON to Sound objects with FIXED preview access
            sounds = []
            for sound_data in data.get('results', []):
                try:
                    # FIX: Use correct key name for preview URL
                    preview_url = sound_data['previews'].get('preview-hq-mp3')

                    if not preview_url:
                        continue

                    sound = type('Sound', (), {
                        'id': sound_data['id'],
                        'name': sound_data['name'],
                        'preview_url': preview_url,
                        'duration': sound_data['duration'],
                        'license': sound_data['license'],
                        'username': sound_data['username']
                    })()
                    sounds.append(sound)
                except (KeyError, TypeError):
                    continue

            return sounds

        except Exception as e:
            print(f"❌ Search error for '{query}': {e}")
            return None

    def download_sound(self, sound, category_path, index):
        """Download sound"""
        try:
            if sound.id in self.downloaded_ids:
                self.stats['skipped_duplicates'] += 1
                return False

            preview_url = sound.preview_url
            response = requests.get(preview_url, timeout=30)
            response.raise_for_status()

            filename = f"{category_path.name}_{index:04d}_id{sound.id}.mp3"
            filepath = category_path / filename

            with open(filepath, 'wb') as f:
                f.write(response.content)

            self.stats['licensing_info'].append({
                'file': filename,
                'sound_id': sound.id,
                'license': sound.license,
                'username': sound.username,
                'duration': sound.duration,
                'freesound_url': f"https://freesound.org/people/{sound.username}/sounds/{sound.id}/"
            })

            self.downloaded_ids.add(sound.id)
            self.stats['successful_downloads'] += 1
            return True

        except Exception as e:
            self.stats['failed_downloads'] += 1
            return False

    def scrape_category(self, category_name, config):
        """Scrape one category"""
        category_path = Path(self.base_path) / category_name
        category_path.mkdir(parents=True, exist_ok=True)

        self.stats['categories'][category_name] = {'queries': config['queries'], 'downloaded': 0}

        print(f"\n{'='*70}")
        print(f"📂 CATEGORY: {category_name.upper()}")
        print(f"{'='*70}")

        download_index = 0

        for query in config['queries']:
            print(f"🔍 Searching: '{query}'")
            results = self.search_sounds(
                query=query,
                max_results=config['target_per_query'],
                duration_range=config['duration_range']
            )

            if results is None or len(results) == 0:
                print(f"   ⚠️  No results found")
                continue

            print(f"   ✓ Found {len(results)} results")
            pbar = tqdm(results, desc=f"Downloading {query[:20]}")

            for sound in pbar:
                success = self.download_sound(sound, category_path, download_index)
                if success:
                    download_index += 1
                    self.stats['categories'][category_name]['downloaded'] += 1

                time.sleep(random.uniform(0.5, 1.5))

        print(f"✓ Downloaded: {self.stats['categories'][category_name]['downloaded']} sounds")

    def scrape_all(self, categories_config):
        """Scrape all categories"""
        print("="*70)
        print("🎬 VIOLENT AUDIO DATASET COLLECTOR")
        print("="*70)
        print(f"📁 Saving to: {self.base_path}")
        print(f"📊 Categories: {len(categories_config)}")
        print("="*70)

        for category_name, config in categories_config.items():
            self.scrape_category(category_name, config)

        self.finalize()

    def finalize(self):
        """Save stats"""
        self.stats['collection_end'] = datetime.now().isoformat()

        stats_file = Path(self.base_path) / 'collection_stats.json'
        with open(stats_file, 'w') as f:
            json.dump(self.stats, f, indent=2)

        if self.stats['licensing_info']:
            licensing_df = pd.DataFrame(self.stats['licensing_info'])
            licensing_file = Path(self.base_path) / 'licensing_attribution.csv'
            licensing_df.to_csv(licensing_file, index=False)

        print("\n\n" + "="*70)
        print("📊 COLLECTION COMPLETE!")
        print("="*70)
        print(f"✅ Downloaded: {self.stats['successful_downloads']}")
        print(f"❌ Failed: {self.stats['failed_downloads']}")
        print(f"⊘ Duplicates: {self.stats['skipped_duplicates']}")
        print("="*70)

        print("\n📋 Per-Category:")
        total_downloaded = 0
        for cat, stats in self.stats['categories'].items():
            print(f"  {cat}: {stats['downloaded']} sounds")
            total_downloaded += stats['downloaded']

        print(f"\n📊 TOTAL AUDIO FILES: {total_downloaded}")
        print(f"📁 Files saved to: {self.base_path}")

# ============================================================================
# RUN THE SCRAPER
# ============================================================================

print("\n" + "="*70)
print("⚙️  CHECKING API KEY...")
print("="*70)

if FREESOUND_API_KEY == "YOUR_API_KEY_HERE":
    print("\n❌ ERROR: Please set your Freesound API key!")
    print("\n📝 Steps:")
    print("   1. Go to: https://freesound.org/apiv2/apply/")
    print("   2. Create/use your API credentials")
    print("   3. Copy your API Key (NOT Client ID)")
    print("   4. Replace 'YOUR_API_KEY_HERE' with your actual key")
    print("   5. Run this cell again")
else:
    print("✅ API key found!")
    print("\n⏳ Starting collection (20-40 minutes)...")
    print("   Keep this notebook open\n")

    try:
        scraper = ViolentAudioScraper(api_key=FREESOUND_API_KEY, base_path=BASE_PATH)
        scraper.scrape_all(VIOLENT_AUDIO_CATEGORIES)

        print("""

✅ SUCCESS! Your dataset is ready.

📂 Dataset Structure:
   violent_audio/
   ├── screams_human_distress/
   ├── gunshots_firearms/
   ├── glass_breaking/
   ├── explosions_blasts/
   ├── sirens_alarms/
   ├── fighting_impact/
   ├── aggressive_speech/
   ├── crash_collision/
   ├── collection_stats.json
   └── licensing_attribution.csv

📥 Download to Your Computer:
   from google.colab import files
   !zip -r violent_audio_dataset.zip /content/datasets/violent_audio
   files.download('violent_audio_dataset.zip')

⚖️  IMPORTANT: Check licensing_attribution.csv for attribution!

🎯 NEXT STEPS:
1. Download your dataset
2. Preprocess audio (16kHz, mono)
3. Extract features (MFCCs, spectrograms)
4. Combine with your image dataset
5. Train multimodal violence detection model!

📊 Expected Performance:
   - Single modality: 70-75% accuracy
   - Audio + Image fusion: 78-85% accuracy
        """)

    except Exception as e:
        print(f"\n❌ Error: {e}")
        import traceback
        traceback.print_exc()


📦 Installing packages...
✅ Packages installed

✅ All modules imported successfully!

✓ Dataset location: /content/datasets/violent_audio


⚙️  CHECKING API KEY...
✅ API key found!

⏳ Starting collection (20-40 minutes)...
   Keep this notebook open

🎬 VIOLENT AUDIO DATASET COLLECTOR
📁 Saving to: /content/datasets/violent_audio
📊 Categories: 8

📂 CATEGORY: SCREAMS_HUMAN_DISTRESS
🔍 Searching: 'scream'
   ✓ Found 50 results


🔍 Searching: 'screaming'
   ✓ Found 50 results


🔍 Searching: 'yell'
   ✓ Found 50 results


🔍 Searching: 'distress'
   ✓ Found 50 results


✓ Downloaded: 151 sounds

📂 CATEGORY: GUNSHOTS_FIREARMS
🔍 Searching: 'gunshot'
❌ Search error for 'gunshot': HTTPSConnectionPool(host='freesound.org', port=443): Read timed out. (read timeout=10)
   ⚠️  No results found
🔍 Searching: 'gun'
❌ Search error for 'gun': HTTPSConnectionPool(host='freesound.org', port=443): Read timed out. (read timeout=10)
   ⚠️  No results found
🔍 Searching: 'pistol'
   ✓ Found 50 results


...

✓ Downloaded: 76 sounds

📂 CATEGORY: CRASH_COLLISION
🔍 Searching: 'crash'
   ✓ Found 25 results


🔍 Searching: 'collision'
   ✓ Found 25 results


🔍 Searching: 'accident'
   ✓ Found 25 results


🔍 Searching: 'car'
   ✓ Found 25 results


✓ Downloaded: 93 sounds


📊 COLLECTION COMPLETE!
✅ Downloaded: 904
❌ Failed: 0
⊘ Duplicates: 196

📋 Per-Category:
  screams_human_distress: 151 sounds
  gunshots_firearms: 91 sounds
  glass_breaking: 129 sounds
  explosions_blasts: 130 sounds
  sirens_alarms: 126 sounds
  fighting_impact: 108 sounds
  aggressive_speech: 76 sounds
  crash_collision: 93 sounds

📊 TOTAL AUDIO FILES: 904
📁 Files saved to: /content/datasets/violent_audio


✅ SUCCESS! Your dataset is ready.

📂 Dataset Structure:
   violent_audio/
   ├── screams_human_distress/
   ├── gunshots_firearms/
   ├── glass_breaking/
   ├── explosions_blasts/
   ├── sirens_alarms/
   ├── fighting_impact/
   ├── aggressive_speech/
   ├── crash_collision/
   ├── collection_stats.json
   └── licensing_attribution.csv

📥 Download to Your Computer:
   from google.colab import files
   !zip -r violent_audio_dataset.zip /content/datasets/violent_audio
   files.download('violent_audio_dataset.zip')

⚖️  IMPORTANT: Check licensing_attribution

In [ ]:
from google.colab import files
!zip -r violent_audio_dataset.zip /content/datasets/violent_audio
files.download('violent_audio_dataset.zip')


  adding: content/datasets/violent_audio/ (stored 0%)
  adding: content/datasets/violent_audio/screams_human_distress/ (stored 0%)
  adding: content/datasets/violent_audio/screams_human_distress/screams_human_distress_0130_id83283.mp3 (deflated 3%)
  adding: content/datasets/violent_audio/screams_human_distress/screams_human_distress_0132_id16090.mp3 (deflated 1%)
  adding: content/datasets/violent_audio/screams_human_distress/screams_human_distress_0022_id520271.mp3 (deflated 2%)
  adding: content/datasets/violent_audio/screams_human_distress/screams_human_distress_0078_id720764.mp3 (deflated 2%)
  adding: content/datasets/violent_audio/screams_human_distress/screams_human_distress_0065_id164024.mp3 (deflated 1%)
  adding: content/datasets/violent_audio/screams_human_distress/screams_human_distress_0072_id179328.mp3 (deflated 1%)
  adding: content/datasets/violent_audio/screams_human_distress/screams_human_distress_0149_id693597.mp3 (deflated 2%)
  adding: content/datasets/violent_aud

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>